In [1]:
#!pip install langchain==0.2.14 langchain-community==0.2.12 langchain-groq===0.1.9 langchain-groq===0.1.9 unstructured==0.14.6 selenium==4.21.0 chromadb==0.5.0 streamlit==1.35.0 pandas==2.0.2 python-dotenv==1.0.0

#link for langchain documentation: https://python.langchain.com/docs/how_to/



In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key ="" #keep you API key here
)

response= llm.invoke("who is the CEO of TELSA INC")
response.content

'The CEO of Tesla, Inc. is Elon Musk.'

In [3]:
response= llm.invoke("who is the CEO of Microsoft INC")
response.content

'The CEO of Microsoft Inc. is Satya Nadella. He has been serving as the CEO since February 4, 2014.'

In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path = "https://jobs.klimb.io/purpletalk/6705034ed20384d491ad6582" #replace with anylink which is available at that moment
)
page_data =loader.load().pop().page_content
print(page_data)



















[x]cube LABS, AI Engineer, Hyderabad, Telangana, India






































									
				        				AI Engineer
									
			        


Hyderabad, Telangana, India 

Permanent

Software Engineering & Technology
2 - 3 years




						View All Jobs
					



								Apply
								



					About us
				 






Share this job





Facebook
Twitter
LinkedIn
Whatsapp



Job Title:


							AI Engineer
						



Job Description:


We are seeking a skilled and innovative AI Engineer with a strong focus on Generative AI and Large Language Models (LLMs) to join our team. The ideal candidate will have deep expertise in AI/ML frameworks, experience working with generative models, and a solid understanding of neural networks and natural language processing (NLP). You will be responsible for building, optimizing, and deploying AI systems, including developing state-of-the-art solutions using tools like Python, PyTorch, TensorFlow, and other advanced AI frameworks.

# make use of chatgpt by giving the job description to create a prompt template preamble like below:

with the help of CHATGPT we can able to extract specific things like in JSON format : role, experience, skills and description

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ##SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTRUCTION
    The scraped text is from a carrer's page of a website.
    your job is to extract the job posting's and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills' and 'description'
    only return the valid JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
response =chain_extract.invoke(input={'page_data': page_data})
print(response.content)
       
       

```
{
  "role": "AI Engineer",
  "experience": "2 - 3 years",
  "skills": [
    "AI/ML",
    "Python",
    "PyTorch",
    "Google TensorFlow",
    "Large Language Models",
    "NLP",
    "Hugging Face",
    "OpenAI API",
    "MLOps",
    "AWS"
  ],
  "description": "We are seeking a skilled and innovative AI Engineer with a strong focus on Generative AI and Large Language Models (LLMs) to join our team. The ideal candidate will have deep expertise in AI/ML frameworks, experience working with generative models, and a solid understanding of neural networks and natural language processing (NLP). You will be responsible for building, optimizing, and deploying AI systems, including developing state-of-the-art solutions using tools like Python, PyTorch, TensorFlow, and other advanced AI frameworks."
}
```


In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response =json_parser.parse(response.content)
json_response

{'role': 'AI Engineer',
 'experience': '2 - 3 years',
 'skills': ['AI/ML',
  'Python',
  'PyTorch',
  'Google TensorFlow',
  'Large Language Models',
  'NLP',
  'Hugging Face',
  'OpenAI API',
  'MLOps',
  'AWS'],
 'description': 'We are seeking a skilled and innovative AI Engineer with a strong focus on Generative AI and Large Language Models (LLMs) to join our team. The ideal candidate will have deep expertise in AI/ML frameworks, experience working with generative models, and a solid understanding of neural networks and natural language processing (NLP). You will be responsible for building, optimizing, and deploying AI systems, including developing state-of-the-art solutions using tools like Python, PyTorch, TensorFlow, and other advanced AI frameworks.'}

In [17]:
type(json_response)

dict

In [18]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="my_portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [19]:
def get_relevant_links(job_skills, collection, max_results=2):
    try:
        query_result = collection.query(
            query_texts=job_skills,
            n_results=max_results
        )
        return query_result.get('metadatas', [])
    except Exception as e:
        print(f"Error querying collection: {e}")
        return []

# Usage
job_skills = job['skills']
relevant_links = get_relevant_links(job_skills, collection)

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'l

In [20]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Warren, a business development executive at SURICA FUSION. SURICA FUSION is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of SURICA FUSION 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Warren, BDE at SURICA FUSION. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert AI Engineer for Your Generative AI and LLM Needs

Dear Hiring Manager,

I came across your job description for an AI Engineer with expertise in Generative AI and Large Language Models (LLMs). I understand that you're looking for a skilled professional to build, optimize, and deploy AI systems using cutting-edge frameworks like PyTorch, TensorFlow, and others.

At SURICA FUSION, we have a proven track record of empowering enterprises with tailored AI solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of expert AI engineers has extensive experience in developing state-of-the-art solutions using Python, PyTorch, TensorFlow, and other advanced AI frameworks.

I'd like to highlight our relevant capabilities in AI/ML and Python development, as demonstrated in our portfolio:

- [https://example.com/ml-python-portfolio](https://example.com/ml-python-portfolio): A comprehensive showcase of our machine learning and 